In [6]:
#C3W1

In [178]:
import os 
import random as rnd

import trax

trax.supervised.training.Loop

import trax.fastmath.numpy as np

from trax import layers as tl

from utilsc3w1 import Layer, load_tweets, process_tweet

In [179]:
# 2. Importing the data

In [180]:
#2.1. Loading the data

In [181]:
import numpy as np

all_positive_tweets, all_negative_tweets = load_tweets()

print(f"The number of positive tweets: {len(all_positive_tweets)}")
print(f"The number of negative tweets: {len(all_negative_tweets)}")

val_pos   = all_positive_tweets[4000:]
train_pos  = all_positive_tweets[:4000]

val_neg   = all_negative_tweets[4000:]
train_neg  = all_negative_tweets[:4000]

train_x = train_pos + train_neg 

val_x  = val_pos + val_neg

train_y = np.append(np.ones(len(train_pos)), np.zeros(len(train_neg)))

val_y  = np.append(np.ones(len(val_pos)), np.zeros(len(val_neg)))

print(f"length of train_x {len(train_x)}")
print(f"length of val_x {len(val_x)}")

The number of positive tweets: 5000
The number of negative tweets: 5000
length of train_x 8000
length of val_x 2000


In [182]:
print("original tweet at training position 0")
print(train_pos[0])

print("Tweet at training position 0 after processing:")
process_tweet(train_pos[0])

original tweet at training position 0
#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)
Tweet at training position 0 after processing:


['followfriday', 'top', 'engag', 'member', 'commun', 'week', ':)']

In [183]:
# 2.2. Building the Vocabulary

In [184]:
Vocab = {'__PAD__': 0, '__</e>__': 1, '__UNK__': 2} 

for tweet in train_x: 
    processed_tweet = process_tweet(tweet)
    for word in processed_tweet:
        if word not in Vocab: 
            Vocab[word] = len(Vocab)
    
print("Total words in vocab are",len(Vocab))
display(Vocab)

Total words in vocab are 9088


{'__PAD__': 0,
 '__</e>__': 1,
 '__UNK__': 2,
 'followfriday': 3,
 'top': 4,
 'engag': 5,
 'member': 6,
 'commun': 7,
 'week': 8,
 ':)': 9,
 'hey': 10,
 'jame': 11,
 'odd': 12,
 ':/': 13,
 'pleas': 14,
 'call': 15,
 'contact': 16,
 'centr': 17,
 '02392441234': 18,
 'abl': 19,
 'assist': 20,
 'mani': 21,
 'thank': 22,
 'listen': 23,
 'last': 24,
 'night': 25,
 'bleed': 26,
 'amaz': 27,
 'track': 28,
 'scotland': 29,
 'congrat': 30,
 'yeaaah': 31,
 'yipppi': 32,
 'accnt': 33,
 'verifi': 34,
 'rqst': 35,
 'succeed': 36,
 'got': 37,
 'blue': 38,
 'tick': 39,
 'mark': 40,
 'fb': 41,
 'profil': 42,
 '15': 43,
 'day': 44,
 'one': 45,
 'irresist': 46,
 'flipkartfashionfriday': 47,
 'like': 48,
 'keep': 49,
 'love': 50,
 'custom': 51,
 'wait': 52,
 'long': 53,
 'hope': 54,
 'enjoy': 55,
 'happi': 56,
 'friday': 57,
 'lwwf': 58,
 'second': 59,
 'thought': 60,
 '’': 61,
 'enough': 62,
 'time': 63,
 'dd': 64,
 'new': 65,
 'short': 66,
 'enter': 67,
 'system': 68,
 'sheep': 69,
 'must': 70,
 'buy':

In [185]:
# 2.3. Converting a tweet to a tensor

In [186]:
def tweet_to_tensor(tweet, vocab_dict, unk_token='__UNK__', verbose=False):
    word_l = process_tweet(tweet)
    
    if verbose:
        print("List of words from the processed tweet:")
        print(word_l)
        
    tensor_l = []
    
    unk_ID = vocab_dict[unk_token]
    
    if verbose:
        print(f"The unique integer ID for the unk_token is {unk_ID}")
        
    for word in word_l:
        word_ID = vocab_dict[word] if word in vocab_dict else unk_ID
        
        tensor_l.append(word_ID) 
    
    return tensor_l

In [187]:
print("Actual tweet is\n", val_pos[0])
print("\nTensor of tweet:\n", tweet_to_tensor(val_pos[0], vocab_dict=Vocab))

Actual tweet is
 Bro:U wan cut hair anot,ur hair long Liao bo
Me:since ord liao,take it easy lor treat as save $ leave it longer :)
Bro:LOL Sibei xialan

Tensor of tweet:
 [1065, 136, 479, 2351, 745, 8148, 1123, 745, 53, 2, 2672, 791, 2, 2, 349, 601, 2, 3489, 1017, 597, 4559, 9, 1065, 157, 2, 2]


In [188]:
# test tweet_to_tensor

def test_tweet_to_tensor():
    test_cases = [
        
        {
            "name":"simple_test_check",
            "input": [val_pos[1], Vocab],
            "expected":[444, 2, 304, 567, 56, 9],
            "error":"The function gives bad output for val_pos[1]. Test failed"
        },
        {
            "name":"datatype_check",
            "input":[val_pos[1], Vocab],
            "expected":type([]),
            "error":"Datatype mismatch. Need only list not np.array"
        },
        {
            "name":"without_unk_check",
            "input":[val_pos[1], Vocab],
            "expected":6,
            "error":"Unk word check not done- Please check if you included mapping for unknown word"
        }
    ]
    count = 0
    for test_case in test_cases:
        
        try:
            if test_case['name'] == "simple_test_check":
                assert test_case["expected"] == tweet_to_tensor(*test_case['input'])
                count += 1
            if test_case['name'] == "datatype_check":
                assert isinstance(tweet_to_tensor(*test_case['input']), test_case["expected"])
                count += 1
            if test_case['name'] == "without_unk_check":
                assert None not in tweet_to_tensor(*test_case['input'])
                count += 1
                
            
            
        except:
            print(test_case['error'])
    if count == 3:
        print("\033[92m All tests passed")
    else:
        print(count," Tests passed out of 3")
test_tweet_to_tensor()            

 All tests passed


In [189]:
# 2.4. Creating a Batch Generator

In [190]:
def data_generator(data_pos, data_neg, batch_size, loop, vocab_dict, shuffle=False):

    assert batch_size % 2 == 0

    n_to_take = batch_size // 2

    pos_index = 0
    neg_index = 0
    
    len_data_pos = len(data_pos)
    len_data_neg = len(data_neg)
    
    pos_index_lines = list(range(len_data_pos))
    neg_index_lines = list(range(len_data_neg))
    
    if shuffle:
        rnd.shuffle(pos_index_lines)
        rnd.shuffle(neg_index_lines)
        
    stop = False
    
    while not stop:  
        
        batch = []

        for i in range(n_to_take):
                    
            if pos_index >= len_data_pos: 
                
                if not loop:
                    stop = True;
                    break;
                
                pos_index = 0
                
                if shuffle:
                    rnd.shuffle(pos_index_lines)
                    
            tweet = data_pos[pos_index_lines[pos_index]]
            
            tensor = tweet_to_tensor(tweet, vocab_dict)
            
            batch.append(tensor)
            
            pos_index = pos_index + 1

        for i in range(n_to_take):            
            if neg_index >= len_data_neg:                
                if not loop:
                    stop = True;
                    break;                    
                neg_index = 0
                
                if shuffle:
                    rnd.shuffle(neg_index_lines)
            tweet = data_neg[neg_index_lines[neg_index]]            
            tensor = tweet_to_tensor(tweet, vocab_dict)
            
            batch.append(tensor)
            
            neg_index += 1

        if stop:
            break;

        pos_index += n_to_take

        neg_index += n_to_take

        max_len = max([len(t) for t in batch]) 

        tensor_pad_l = []
        for tensor in batch:

            n_pad = max_len - len(tensor)
            
            pad_l = [0]*n_pad
            
            tensor_pad = tensor + pad_l
            
            tensor_pad_l.append(tensor_pad)

        inputs = np.array(tensor_pad_l)

        target_pos = [1]*n_to_take
        target_neg = [0]*n_to_take
        
        target_l = target_pos + target_neg
        targets = np.array(target_l)

        example_weights = np.ones_like(targets)

        yield inputs, targets, example_weights

In [191]:
# Set the random number generator for the shuffle procedure
rnd.seed(30) 

def train_generator(batch_size, shuffle = False):
    return data_generator(train_pos, train_neg, batch_size, True, Vocab, shuffle)

def val_generator(batch_size, shuffle = False):
    return data_generator(val_pos, val_neg, batch_size, True, Vocab, shuffle)

def test_generator(batch_size, shuffle = False):
    return data_generator(val_pos, val_neg, batch_size, False, Vocab, shuffle)

inputs, targets, example_weights = next(train_generator(4, shuffle=True))

print(f'Inputs: {inputs}')
print(f'Targets: {targets}')
print(f'Example Weights: {example_weights}')

Inputs: [[2005 4451 3201    9    0    0    0    0    0    0    0]
 [4954  567 2000 1454 5174 3499  141 3499  130  459    9]
 [3761  109  136  583 2930 3969    0    0    0    0    0]
 [ 250 3761    0    0    0    0    0    0    0    0    0]]
Targets: [1 1 0 0]
Example Weights: [1 1 1 1]


In [192]:
# Test the train_generator

tmp_data_gen = train_generator(batch_size = 4)

tmp_inputs, tmp_targets, tmp_example_weights = next(tmp_data_gen)

print(f"The inputs shape is {tmp_inputs.shape}")
print(f"The targets shape is {tmp_targets.shape}")
print(f"The example weights shape is {tmp_example_weights.shape}")

for i,t in enumerate(tmp_inputs):
    print(f"input tensor: {t}; target {tmp_targets[i]}; example weights {tmp_example_weights[i]}")

The inputs shape is (4, 14)
The targets shape is (4,)
The example weights shape is (4,)
input tensor: [3 4 5 6 7 8 9 0 0 0 0 0 0 0]; target 1; example weights 1
input tensor: [10 11 12 13 14 15 16 17 18 19 20  9 21 22]; target 1; example weights 1
input tensor: [5738 2901 3761    0    0    0    0    0    0    0    0    0    0    0]; target 0; example weights 1
input tensor: [ 858  256 3652 5739  307 4458  567 1230 2767  328 1202 3761    0    0]; target 0; example weights 1


In [193]:
# 3. Defining Classes

In [194]:
# 3.1. Relu Class

In [195]:

class Relu(Layer):
    def forward(self, x):
        
        activation = np.maximum(x,0)
        
        return activation

In [196]:
# Test your relu function
x = np.array([[-2.0, -1.0, 0.0], [0.0, 1.0, 2.0]], dtype=float)
relu_layer = Relu()
print("Test data is:")
print(x)
print("Output of Relu is:")
print(relu_layer(x))

Test data is:
[[-2. -1.  0.]
 [ 0.  1.  2.]]
Output of Relu is:
[[0. 0. 0.]
 [0. 1. 2.]]


In [197]:
# 3.2. Dense Class

In [198]:
from trax import fastmath

np = fastmath.numpy
random = fastmath.random

In [199]:
# See how the fastmath.trax.random.normal function works
tmp_key = random.get_prng(seed=1)
print("The random seed generated by random.get_prng")
display(tmp_key)

print("choose a matrix with 2 rows and 3 columns")
tmp_shape=(2,3)
display(tmp_shape)

tmp_weight = trax.fastmath.random.normal(key=tmp_key, shape=tmp_shape)

print("Weight matrix generated with a normal distribution with mean 0 and stdev of 1")
display(tmp_weight)

The random seed generated by random.get_prng


Buffer([0, 1], dtype=uint32)

choose a matrix with 2 rows and 3 columns


(2, 3)

Weight matrix generated with a normal distribution with mean 0 and stdev of 1


Buffer([[ 0.957307  , -0.9699291 ,  1.0070664 ],
        [ 0.36619025,  0.17294823,  0.29092228]], dtype=float32)

In [200]:
class Dense(Layer):

    def __init__(self, n_units, init_stdev=0.1):        
        self._n_units = n_units
        self._init_stdev = 0.1

    def forward(self, x):
        dense = np.dot(x, self.weights) 
        return dense

    # init_weights
    def init_weights_and_state(self, input_signature, random_key):
        input_shape = input_signature.shape      
        w = self._init_stdev * random.normal(
            key = random_key, shape = (input_shape[-1], self._n_units))
           
        self.weights = w
        return self.weights

In [201]:
# Testing your Dense layer 
dense_layer = Dense(n_units=10)  
random_key = random.get_prng(seed=0)
z = np.array([[2.0, 7.0, 25.0]])

dense_layer.init(z, random_key)
print("Weights are\n ",dense_layer.weights)
print("Foward function output is ", dense_layer(z))

Weights are
  [[-0.02837108  0.09368161 -0.10050074  0.14165013  0.105433    0.09108126
  -0.04265672  0.0986188  -0.05575325  0.00153249]
 [-0.20785688  0.0554837   0.09142365  0.05744595  0.07227863  0.01210617
  -0.03237354  0.16234994  0.02450038 -0.13809782]
 [-0.06111238  0.01403724  0.08410042 -0.1094358  -0.10775021 -0.11396459
  -0.05933381 -0.01557652 -0.03832145 -0.11144515]]
Foward function output is  [[-3.0395498   0.92668015  2.5414746  -2.050473   -1.976939   -2.582209
  -1.7952734   0.9442741  -0.8980402  -3.7497485 ]]


In [202]:
# 3.3. Model

In [203]:
tmp_embed = tl.Embedding(vocab_size=3, d_feature=2)
display(tmp_embed)

Embedding_3_2

In [204]:
tmp_embed = np.array([[1,2,3,],
                    [4,5,6]
                   ])

print("The mean along axis 0 creates a vector whose length equals the vocabulary size")
display(np.mean(tmp_embed,axis=0))

print("The mean along axis 1 creates a vector whose length equals the number of elements in a word embedding")
display(np.mean(tmp_embed,axis=1))

The mean along axis 0 creates a vector whose length equals the vocabulary size


DeviceArray([2.5, 3.5, 4.5], dtype=float32)

The mean along axis 1 creates a vector whose length equals the number of elements in a word embedding


DeviceArray([2., 5.], dtype=float32)

In [205]:
def classifier(vocab_size=len(Vocab), embedding_dim=256, output_dim=2, mode='train'):

    embed_layer = tl.Embedding(
        vocab_size=vocab_size, 
        d_feature=embedding_dim)
    
    mean_layer = tl.Mean(axis=1)
    
    dense_output_layer = tl.Dense(n_units = output_dim)

    
    log_softmax_layer = tl.LogSoftmax()

    model = tl.Serial(
      embed_layer,
      mean_layer,
      dense_output_layer,
      log_softmax_layer
    )
    return model

In [206]:
tmp_model = classifier()

In [207]:
print(type(tmp_model))
display(tmp_model)

<class 'trax.layers.combinators.Serial'>


Serial[
  Embedding_9088_256
  Mean
  Dense_2
  LogSoftmax
]

In [208]:
# 4.1. Training the model

In [209]:
from trax.supervised import training

batch_size = 16
rnd.seed(271)

train_task = training.TrainTask(
    labeled_data=train_generator(batch_size=batch_size, shuffle=True),
    loss_layer=tl.CrossEntropyLoss(),
    optimizer=trax.optimizers.Adam(0.01),
    n_steps_per_checkpoint=10,
)

eval_task = training.EvalTask(
    labeled_data=val_generator(batch_size=batch_size, shuffle=True),
    metrics=[tl.CrossEntropyLoss(), tl.Accuracy()],
)

model = classifier()

In [210]:
output_dir = '~/model/'
output_dir_expand = os.path.expanduser(output_dir)
print(output_dir_expand)

/Users/diegobatalhacunhadasilva/model/


In [211]:
def train_model(classifier, train_task, eval_task, n_steps, output_dir):

    training_loop = training.Loop(
                                classifier,
                                train_task,
                                eval_tasks = eval_task,
                                output_dir = output_dir)

    training_loop.run(n_steps = n_steps)

    return training_loop

In [212]:
training_loop = train_model(model, train_task, eval_task, 100, output_dir_expand)


Step    310: Ran 10 train steps in 12.07 secs
Step    310: train CrossEntropyLoss |  0.00130165
Step    310: eval  CrossEntropyLoss |  0.00011680
Step    310: eval          Accuracy |  1.00000000

Step    320: Ran 10 train steps in 6.48 secs
Step    320: train CrossEntropyLoss |  0.00039270
Step    320: eval  CrossEntropyLoss |  0.01050072
Step    320: eval          Accuracy |  1.00000000

Step    330: Ran 10 train steps in 4.30 secs
Step    330: train CrossEntropyLoss |  0.00074696
Step    330: eval  CrossEntropyLoss |  0.00194571
Step    330: eval          Accuracy |  1.00000000

Step    340: Ran 10 train steps in 2.56 secs
Step    340: train CrossEntropyLoss |  0.01558403
Step    340: eval  CrossEntropyLoss |  0.00391711
Step    340: eval          Accuracy |  1.00000000

Step    350: Ran 10 train steps in 1.20 secs
Step    350: train CrossEntropyLoss |  0.00877521
Step    350: eval  CrossEntropyLoss |  0.01291416
Step    350: eval          Accuracy |  1.00000000

Step    360: Ran 1

In [213]:
# 4.2. Practice Making a Prediction

In [214]:
tmp_train_generator = train_generator(16)

tmp_batch = next(tmp_train_generator)

tmp_inputs, tmp_targets, tmp_example_weights = tmp_batch

print(f"The batch is a tuple of length {len(tmp_batch)} because position 0 contains the tweets, and position 1 contains the targets.") 
print(f"The shape of the tweet tensors is {tmp_inputs.shape} (num of examples, length of tweet tensors)")
print(f"The shape of the labels is {tmp_targets.shape}, which is the batch size.")
print(f"The shape of the example_weights is {tmp_example_weights.shape}, which is the same as inputs/targets size.")

The batch is a tuple of length 3 because position 0 contains the tweets, and position 1 contains the targets.
The shape of the tweet tensors is (16, 15) (num of examples, length of tweet tensors)
The shape of the labels is (16,), which is the batch size.
The shape of the example_weights is (16,), which is the same as inputs/targets size.


In [215]:
tmp_pred = training_loop.eval_model(tmp_inputs)
print(f"The prediction shape is {tmp_pred.shape}, num of tensor_tweets as rows")
print("Column 0 is the probability of a negative sentiment (class 0)")
print("Column 1 is the probability of a positive sentiment (class 1)")
print()
print("View the prediction array")
tmp_pred

The prediction shape is (16, 2), num of tensor_tweets as rows
Column 0 is the probability of a negative sentiment (class 0)
Column 1 is the probability of a positive sentiment (class 1)

View the prediction array


DeviceArray([[-1.0037046e+01, -4.3869019e-05],
             [-7.4305410e+00, -5.9294701e-04],
             [-1.0318024e+01, -3.2901764e-05],
             [-1.0070904e+01, -4.2438507e-05],
             [-1.0103464e+01, -4.1007996e-05],
             [-9.8432789e+00, -5.2928925e-05],
             [-1.2162621e+01, -5.2452087e-06],
             [-5.3617773e+00, -4.7037601e-03],
             [-9.5367432e-07, -1.3715170e+01],
             [ 0.0000000e+00, -1.5701397e+01],
             [ 0.0000000e+00, -1.5249322e+01],
             [ 0.0000000e+00, -2.6646961e+01],
             [-3.7455559e-04, -7.8900232e+00],
             [-1.4305115e-06, -1.3485584e+01],
             [-1.4305115e-06, -1.3555519e+01],
             [-4.7683716e-07, -1.4546761e+01]], dtype=float32)

In [216]:
tmp_is_positive = tmp_pred[:,1] > tmp_pred[:,0]
for i, p in enumerate(tmp_is_positive):
    print(f"Neg log prob {tmp_pred[i,0]:.4f}\tPos log prob {tmp_pred[i,1]:.4f}\t is positive? {p}\t actual {tmp_targets[i]}")

Neg log prob -10.0370	Pos log prob -0.0000	 is positive? True	 actual 1
Neg log prob -7.4305	Pos log prob -0.0006	 is positive? True	 actual 1
Neg log prob -10.3180	Pos log prob -0.0000	 is positive? True	 actual 1
Neg log prob -10.0709	Pos log prob -0.0000	 is positive? True	 actual 1
Neg log prob -10.1035	Pos log prob -0.0000	 is positive? True	 actual 1
Neg log prob -9.8433	Pos log prob -0.0001	 is positive? True	 actual 1
Neg log prob -12.1626	Pos log prob -0.0000	 is positive? True	 actual 1
Neg log prob -5.3618	Pos log prob -0.0047	 is positive? True	 actual 1
Neg log prob -0.0000	Pos log prob -13.7152	 is positive? False	 actual 0
Neg log prob 0.0000	Pos log prob -15.7014	 is positive? False	 actual 0
Neg log prob 0.0000	Pos log prob -15.2493	 is positive? False	 actual 0
Neg log prob 0.0000	Pos log prob -26.6470	 is positive? False	 actual 0
Neg log prob -0.0004	Pos log prob -7.8900	 is positive? False	 actual 0
Neg log prob -0.0000	Pos log prob -13.4856	 is positive? False	 ac

In [217]:
# View the array of booleans
print("Array of booleans")
display(tmp_is_positive)

# convert boolean to type int32
# True is converted to 1
# False is converted to 0
tmp_is_positive_int = tmp_is_positive.astype(np.int32)


# View the array of integers
print("Array of integers")
display(tmp_is_positive_int)

# convert boolean to type float32
tmp_is_positive_float = tmp_is_positive.astype(np.float32)

# View the array of floats
print("Array of floats")
display(tmp_is_positive_float)

Array of booleans


DeviceArray([ True,  True,  True,  True,  True,  True,  True,  True,
             False, False, False, False, False, False, False, False],            dtype=bool)

Array of integers


DeviceArray([1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Array of floats


DeviceArray([1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
             0.], dtype=float32)

In [218]:
tmp_pred.shape

(16, 2)

In [219]:
print(f"True == 1: {True == 1}")
print(f"True == 2: {True == 2}")
print(f"False == 0: {False == 0}")
print(f"False == 2: {False == 2}")

True == 1: True
True == 2: False
False == 0: True
False == 2: False


In [220]:
# 5. Evaluation

In [221]:
# 5.1. Computing the accuracity of the batch

In [222]:
def compute_accuracy(preds, y, y_weights):

    is_pos =  preds[:, 1] > preds[:, 0] 

    is_pos_int = is_pos.astype(np.int32)
    
    correct = is_pos_int == y 

    sum_weights = np.sum(y_weights)
    
    correct_float = correct.astype(np.float32)
    
    weighted_correct_float = correct_float * y_weights
    weighted_num_correct = np.sum(weighted_correct_float)
 
    accuracy = weighted_num_correct / sum_weights

    return accuracy, weighted_num_correct, sum_weights

In [223]:
# test your function
tmp_val_generator = val_generator(64)

# get one batch
tmp_batch = next(tmp_val_generator)

# Position 0 has the model inputs (tweets as tensors)
# position 1 has the targets (the actual labels)
tmp_inputs, tmp_targets, tmp_example_weights = tmp_batch

# feed the tweet tensors into the model to get a prediction
tmp_pred = training_loop.eval_model(tmp_inputs)

tmp_acc, tmp_num_correct, tmp_num_predictions = compute_accuracy(preds=tmp_pred, y=tmp_targets, y_weights=tmp_example_weights)

print(f"Model's prediction accuracy on a single training batch is: {100 * tmp_acc}%")
print(f"Weighted number of correct predictions {tmp_num_correct}; weighted number of total observations predicted {tmp_num_predictions}")

Model's prediction accuracy on a single training batch is: 100.0%
Weighted number of correct predictions 64.0; weighted number of total observations predicted 64


In [224]:
# 5.2. Testing Your Model on Validation data

In [225]:

def test_model(generator, model):
    
    accuracy = 0.
    total_num_correct = 0
    total_num_pred = 0
    
    for batch in generator: 
        
        inputs = batch[0]
        
        targets = batch[1]
        
        example_weight = batch[2]

        pred = model(inputs)
        
        batch_accuracy, batch_num_correct, batch_num_pred = compute_accuracy(pred, targets, example_weight) 

        total_num_correct += batch_num_correct

        total_num_pred += batch_num_pred

    accuracy = total_num_correct / total_num_pred
    
    return accuracy

In [226]:
model = training_loop.eval_model
accuracy = test_model(test_generator(16), model)

print(f'The accuracy of your model on the validation set is {accuracy:.4f}', )

The accuracy of your model on the validation set is 0.9940


In [227]:
# 6. Testing with our own input

In [228]:
def predict(sentence):
    inputs = np.array(tweet_to_tensor(sentence, vocab_dict=Vocab))
    
    inputs = inputs[None, :]  
    
    preds_probs = model(inputs)
    
    preds = int(preds_probs[0, 1] > preds_probs[0, 0])
    
    sentiment = "negative"
    if preds == 1:
        sentiment = 'positive'

    return preds, sentiment


In [229]:
sentence = "It's such a nice day, think i'll be taking Sid to Ramsgate fish and chips for lunch at Peter's fish factory and then the beach maybe"
tmp_pred, tmp_sentiment = predict(sentence)
print(f"The sentiment of the sentence \n***\n\"{sentence}\"\n***\nis {tmp_sentiment}.")

print()
sentence = "I hated my day, it was the worst, I'm so sad."
tmp_pred, tmp_sentiment = predict(sentence)
print(f"The sentiment of the sentence \n***\n\"{sentence}\"\n***\nis {tmp_sentiment}.")

The sentiment of the sentence 
***
"It's such a nice day, think i'll be taking Sid to Ramsgate fish and chips for lunch at Peter's fish factory and then the beach maybe"
***
is positive.

The sentiment of the sentence 
***
"I hated my day, it was the worst, I'm so sad."
***
is negative.
